In [31]:
import pymupdf

doc = pymupdf.open("iesc111.pdf") # open a document
text_data = ''
for page in doc: # iterate the document pages
    text = page.get_text()
    text_data += text

In [32]:
print(text_data[:100])

Everyday we hear sounds from various
sources like humans, birds, bells, machines,
vehicles, televisi


In [33]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")
encoded = encoding.encode('My name is Divya Gupta')
encoded

[5159, 836, 374, 8940, 7911, 74798]

In [34]:
encoded = encoding.encode(text_data)

len(encoded)

8253

the whole book context is far les than the context window(128k) of almost all newer llms, and is also around the limit for embedding models(8K)

In [67]:
# i have broken the book into segments of ~1000 tokens each (in hope that each segemnt would have a complete sense of its own)
segments = []
segment = ''
for line in text_data.split('\n'):
    segment += '\n' + line
    if len(encoding.encode(segment)) > 1000:
        segments.append(segment)
        segment = line
segments.append(segment)

len(segments)

9

In [55]:
# sanity check  
print(segments[0][0:100])


Everyday we hear sounds from various
sources like humans, birds, bells, machines,
vehicles, televis


In [57]:
from openai import OpenAI
client = OpenAI(api_key='YOUR_OPENAI_API_KEY')

def get_embeddings(tokens):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=tokens
    )
    return response.data[0].embedding

In [58]:
embeddings = []
for i, segment in enumerate(segments):
    print(f'Processing segment {i}')
    embeddings.append(get_embeddings(segment))

Processing segment 0
Processing segment 1
Processing segment 2
Processing segment 3
Processing segment 4
Processing segment 5
Processing segment 6
Processing segment 7
Processing segment 8


In [59]:
from pinecone import Pinecone

pc = Pinecone(api_key="d72b82f0-748c-4b64-9ba1-424e87aa5c36")
index = pc.Index("sarvam-assignment")

In [60]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [62]:
i = 1
for embedding, segment in zip(embeddings, segments):
    index.upsert(
        vectors=[
            {
                "id": str(i), 
                "values": embedding,
                "metadata": {"text": segment}
            }
        ],
    )
    i += 1

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [63]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 9}},
 'total_vector_count': 9}

In [65]:
index.query(
    vector=embeddings[0],
    top_k=3,
    include_values=False,
    include_metadata=True,
)

{'matches': [{'id': '1',
              'metadata': {'text': '\n'
                                   'Everyday we hear sounds from various\n'
                                   'sources like humans, birds, bells, '
                                   'machines,\n'
                                   'vehicles, televisions, radios etc. Sound '
                                   'is a\n'
                                   'form of energy which produces a sensation\n'
                                   'of hearing in our ears. There are also '
                                   'other\n'
                                   'forms of energy like mechanical energy, '
                                   'light\n'
                                   'energy, etc. We have talked about '
                                   'mechanical\n'
                                   'energy in the previous chapters. You have\n'
                                   'been taught about conservation of energy,\n'
    